In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import *
import re
from matplotlib import pyplot as plt
import seaborn as sns
import sqlite3

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [2]:
DB = "..\\Data\\hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

<h2>Stat Functions</h2>
<p>Given the player statistics calculate:
    <ul>
        <li>Age</li>
        <li>Total minutes this season</li>
        <li>Total minutes last 3 weeks</li>
        <li>shots_against this season</li>
        <li>shots_against 3 weeks</li>
        <li>save_pct this season</li>
        <li>save_pct 3 weeks</li>
        <li>rest days</li>
       
</ul></p>

In [14]:
def season(date):
    if date.month > 8:
        return date.year
    else:
        return date.year-1
def cumulative_stats(dfrow):
    """
    
        
    """
    #minutes_season 
    dfrow['']
    pass

In [31]:
player_data = pd.DataFrame(columns = ['age','injured'])

q = """SELECT * FROM player_log"""
player_logs = run_query(q)
player_logs['date_game'] = player_logs['date_game'].astype('datetime64')
#convert time on ice to float
player_logs['time_on_ice'] = player_logs['time_on_ice'].str.extract(r'(\d*)\:\d*')[0].astype(int)+player_logs['time_on_ice'].str.extract(r'\d*\:(\d*)')[0].astype(int)/60
player_logs['season'] = player_logs['date_game'].map(lambda x: season(x))

player_list = player_logs['player_id'].unique()
for player in player_list[:1]:
    plogs = player_logs[player_logs['player_id']==player]
    

aebisda01


In [35]:
plogs = player_logs[player_logs['player_id']==player].reset_index(drop=True)
plogs['total_min'] = 0
plogs['min_3weeks'] = 0
for i,row in plogs.iterrows

,gamelog_id,player_id,date_game,game_season,age,team_id,game_location,opp_id,game_result,goals_against,shots_against,saves,save_pct,shutouts,pen_min,time_on_ice,teamlog_id,injured,season
0,aebisda0120001018,aebisda01,2000-10-18,1,22.696,COL,@,CBJ,W,1,21,20,0.952,0,0,60.000000,None,0,2000
1,aebisda0120001026,aebisda01,2000-10-26,2,22.718,COL,@,CHI,W,0,18,18,1.000,1,0,60.000000,None,0,2000
2,aebisda0120001103,aebisda01,2000-11-03,3,22.740,COL,None,CAR,W,3,26,23,0.885,0,0,60.000000,None,0,2000
3,aebisda0120001111,aebisda01,2000-11-11,4,22.762,COL,None,MDA,W,1,19,18,0.947,0,0,60.000000,None,0,2000
4,aebisda0120001116,aebisda01,2000-11-16,5,22.775,COL,@,PHX,L,6,28,22,0.786,0,0,59.916667,None,0,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,aebisda0120070308,aebisda01,2007-03-08,29,29.079,MTL,@,ATL,L,1,14,13,0.929,0,0,33.216667,None,0,2006
210,aebisda0120070310,aebisda01,2007-03-10,30,29.085,MTL,@,STL,W,3,28,25,0.893,0,0,60.000000,None,0,2006
211,aebisda0120070313,aebisda01,2007-03-13,31,29.093,MTL,None,NYI,W,3,27,24,0.889,0,0,59.400000,None,0,2006
212,aebisda0120070316,aebisda01,2007-03-16,32,29.101,MTL,@,PIT,L,5,30,25,0.833,0,0,59.800000,None,0,2006
